In [1]:
!pip install transformers datasets pandas torch openpyxl

In [2]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import Dataset
import numpy as np
import wandb

In [3]:
# Load the dataset
df = pd.read_excel("dataset_rv.xlsx")

# Map string labels to numerical labels
label_map = {"outrage": 0, "despair": 1, "hope": 2}
df["Sentiment"] = df["Sentiment"].map(label_map)

# Rename the column to text
df.rename(columns={'Headline':'text', 'Sentiment':'labels'}, inplace=True)

In [4]:
def english_to_bangla_number_text(number):
    """Converts an English number to Bangla textual representation."""
    bangla_numbers = {
        0: "শূন্য", 1: "এক", 2: "দুই", 3: "তিন", 4: "চার",
        5: "পাঁচ", 6: "ছয়", 7: "সাত", 8: "আট", 9: "নয়",
        10: "দশ", 11: "এগারো", 12: "বারো", 13: "তেরো", 14: "চৌদ্দ",
        15: "পনেরো", 16: "ষোলো", 17: "সতেরো", 18: "আঠারো", 19: "উনিশ",
        20: "বিশ", 21: "একুশ", 22: "বাইশ", 23: "তেইশ", 24: "চব্বিশ",
        25: "পঁচিশ", 26: "ছাব্বিশ", 27: "সাতাশ", 28: "আটাশ", 29: "ঊনত্রিশ",
        30: "ত্রিশ", 31: "একত্রিশ", 32: "বত্রিশ", 33: "তেত্রিশ", 34: "চৌত্রিশ",
        35: "পঁত্রিশ", 36: "ছত্রিশ", 37: "সাঁইত্রিশ", 38: "আটত্রিশ", 39: "ঊনচল্লিশ",
        40: "চল্লিশ", 41: "একচল্লিশ", 42: "বিয়াল্লিশ", 43: "তেতাল্লিশ", 44: "চুয়াল্লিশ",
        45: "পঁইয়াল্লিশ", 46: "ছিয়াল্লিশ", 47: "সাতচল্লিশ", 48: "আটচল্লিশ", 49: "ঊনপঞ্চাশ",
        50: "পঞ্চাশ", 51: "একান্ন", 52: "বাহান্ন", 53: "তিপ্পান্ন", 54: "চুয়ান্ন",
        55: "পঞ্চান্ন", 56: "ছাপ্পান্ন", 57: "সাতান্ন", 58: "আটান্ন", 59: "ঊনষাট",
        60: "ষাট", 61: "একষট্টি", 62: "বাষট্টি", 63: "তেষট্টি", 64: "চৌষট্টি",
        65: "পঁষট্টি", 66: "ছেষট্টি", 67: "সাতষট্টি", 68: "আটষট্টি", 69: "ঊনসত্তর",
        70: "সত্তর", 71: "একাত্তর", 72: "বাহাত্তর", 73: "তিয়াত্তর", 74: "চুয়াত্তর",
        75: "পঁচাত্তর", 76: "ছিয়াত্তর", 77: "সাতাত্তর", 78: "আটাত্তর", 79: "ঊনআশি",
        80: "আশি", 81: "একাশি", 82: "বিরাশি", 83: "তিরাশি", 84: "চুরাশি",
        85: "পঁচাশি", 86: "ছিয়াশি", 87: "সাতাশি", 88: "আটাশি", 89: "ঊননব্বই",
        90: "নব্বই", 91: "একানব্বই", 92: "বিরানব্বই", 93: "তিরানব্বই", 94: "চুরানব্বই",
        95: "পঁচানব্বই", 96: "ছিয়ানব্বই", 97: "সাতানব্বই", 98: "আটানব্বই", 99: "নিরানব্বই",
    }

    def convert_two_digit_number(n):
        """Handles numbers from 0 to 99 based on direct mapping."""
        if n in bangla_numbers:
            return bangla_numbers[n]
        tens = (n // 10) * 10
        units = n % 10
        return f"{bangla_numbers[tens]} {bangla_numbers[units]}" if units > 0 else bangla_numbers[tens]

    if number < 100:
        return convert_two_digit_number(number)
    elif number < 1000:
        hundreds = number // 100
        remainder = number % 100
        if remainder == 0:
            return f"{bangla_numbers[hundreds]} শত"
        return f"{bangla_numbers[hundreds]} শত {convert_two_digit_number(remainder)}"
    elif number < 100000: #Handle numbers from 1000 to 99,999
        parts = []
        if number >= 1000:
            parts.append(f"{english_to_bangla_number_text(number // 1000)} হাজার")
            number %= 1000
        if number > 0:
            if number >= 100:
              parts.append(english_to_bangla_number_text(number))
            else:
              parts.append(convert_two_digit_number(number))  #handles cases less than 100
        return " ".join(parts)
    elif number < 10000000: #handles numbers from 100,000 to 9,999,999
      parts = []
      if number >= 100000:
          parts.append(f"{english_to_bangla_number_text(number // 100000)} লক্ষ")
          number %= 100000
      if number > 0:
          if number >= 1000:
            parts.append(english_to_bangla_number_text(number))
          elif number > 0:
            if number >= 100:
                parts.append(english_to_bangla_number_text(number))
            else:
              parts.append(convert_two_digit_number(number)) #handle numbers less than 100
      return " ".join(parts)
    else: # Handle numbers >= 10,000,000 (Crore)
        parts = []
        if number >= 10000000:
            parts.append(f"{english_to_bangla_number_text(number // 10000000)} কোটি")
            number %= 10000000
        if number > 0:
            parts.append(english_to_bangla_number_text(number))
        return " ".join(parts)

In [5]:
import re

def text_to_word_list(text):
    text = text.split()
    return text

def replace_strings(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\u2600-\u26FF"          # miscellaneous symbols
                               u"\u2700-\u27BF"          # dingbats
                               u"\u2000-\u206F"          # general punctuations
                               "]+", flags=re.UNICODE)
    english_pattern = re.compile('[a-zA-Z]+', flags=re.I)

    text = emoji_pattern.sub(r'', text)
    text = english_pattern.sub(r'', text)

    return text

def remove_punctuations(my_str):
    punctuations = '''````£|¢|Ñ+-*/=EROero৳০১২৩৪৫৬৭৮৯012–34567•89।!()-[]{};:'"“\’,<>./?@#$%^&*_~‘—॥”‰🤣⚽️✌�￰৷￰'''
    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct += char

    return no_punct

def convert_numbers_to_bangla(text):
    words = text.split()
    converted_words = []
    for word in words:
        if word.isdigit():  # Check if the word is an integer
            bangla_number = english_to_bangla_number_text(int(word))
            converted_words.append(bangla_number)
        else:
            converted_words.append(word)
    return ' '.join(converted_words)

def preprocessing(text):
    text = replace_strings(text)
    text = convert_numbers_to_bangla(text)  # Convert numbers to Bangla
    text = remove_punctuations(text)
    return text

In [6]:
df['text'] =df['text'].apply(lambda x: preprocessing(str(x)))
df.head()

,text,labels
0,ঢাকা বিশ্ববিদ্যালয় কোটাবিরোধী আন্দোলনের নেতাকে...,0
1,কোটাবিরোধী আন্দোলনে ঢাকা বিশ্ববিদ্যালয়ের বিএনপ...,2
2,কোটাবিরোধী আন্দোলন আজও জিরো পয়েন্ট অবরোধ করে ...,0
3,সর্বজনীন পেনশন সরকার অনড় আন্দোলন চালিয়ে যাবেন ...,0
4,আজ শনিবার সকাল সাড়ে টার দিকে মহাসড়কের শহর বাইপ...,0


In [7]:
# Split the data
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert to Hugging Face Dataset objects
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [8]:
# Load the model and tokenizer
model_name = "csebuetnlp/banglabert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3) # num_labels=3 (outrage/despair/hope)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at csebuetnlp/banglabert and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1622 [00:00<?, ? examples/s]

Map:   0%|          | 0/406 [00:00<?, ? examples/s]

In [11]:
training_args = TrainingArguments(
    output_dir="./banglabert_sentiment",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Initialize wandb
wandb.init(project="banglabert-sentiment",
             name="banglabert-sentiment-run",
             config=training_args)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: tanvirrahmedd (tanvirrahmedd-north-south-university). Use `wandb login --relogin` to force relogin


In [12]:
def compute_metrics(eval_pred):
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   from sklearn.metrics import accuracy_score, f1_score
   accuracy = accuracy_score(labels, predictions)
   f1 = f1_score(labels, predictions, average='weighted')
   return {"accuracy": accuracy, "f1_score": f1}

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train_dataset,
   eval_dataset=tokenized_test_dataset,
   tokenizer=tokenizer,
   compute_metrics=compute_metrics
)

<ipython-input-12-cac006295104>:9: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [13]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,No log,0.721569,0.741379,0.722974
2,No log,0.591907,0.778325,0.767533
3,No log,0.627684,0.780788,0.778094
4,No log,0.745512,0.790640,0.785824
5,0.444200,0.791546,0.812808,0.811139
6,0.444200,0.939942,0.793103,0.796041
7,0.444200,0.969602,0.807882,0.807001
8,0.444200,0.998437,0.815271,0.813883
9,0.444200,1.059058,0.807882,0.806774
10,0.028400,1.064830,0.820197,0.817990


TrainOutput(global_step=1530, training_loss=0.15580185123907975, metrics={'train_runtime': 2507.7338, 'train_samples_per_second': 9.702, 'train_steps_per_second': 0.61, 'total_flos': 6401549453322240.0, 'train_loss': 0.15580185123907975, 'epoch': 15.0})

In [14]:
evaluation = trainer.evaluate()
print(evaluation)

{'eval_loss': 0.5919071435928345, 'eval_accuracy': 0.7783251231527094, 'eval_f1_score': 0.7675333419094921, 'eval_runtime': 12.8776, 'eval_samples_per_second': 31.528, 'eval_steps_per_second': 2.019, 'epoch': 15.0}


In [15]:
trainer.save_model("./fine_tuned_banglabert_sentiment")
tokenizer.save_pretrained("./fine_tuned_banglabert_sentiment")

('./fine_tuned_banglabert_sentiment/tokenizer_config.json',
 './fine_tuned_banglabert_sentiment/special_tokens_map.json',
 './fine_tuned_banglabert_sentiment/vocab.txt',
 './fine_tuned_banglabert_sentiment/added_tokens.json',
 './fine_tuned_banglabert_sentiment/tokenizer.json')

In [16]:
!zip -r /content/fine_tuned_banglabert_sentiment.zip /content/fine_tuned_banglabert_sentiment


updating: content/fine_tuned_banglabert_sentiment/ (stored 0%)
updating: content/fine_tuned_banglabert_sentiment/training_args.bin (deflated 51%)
updating: content/fine_tuned_banglabert_sentiment/tokenizer.json (deflated 76%)
updating: content/fine_tuned_banglabert_sentiment/config.json (deflated 54%)
updating: content/fine_tuned_banglabert_sentiment/tokenizer_config.json (deflated 74%)
updating: content/fine_tuned_banglabert_sentiment/special_tokens_map.json (deflated 42%)
updating: content/fine_tuned_banglabert_sentiment/vocab.txt (deflated 71%)
updating: content/fine_tuned_banglabert_sentiment/model.safetensors (deflated 7%)


In [17]:
# Load the fine-tuned model and tokenizer
model_path = "./fine_tuned_banglabert_sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=3)
model.eval()  # Set the model to evaluation mode

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [18]:
# Define a reverse label mapping
reverse_label_map = {0: "outrage", 1: "despair", 2: "hope"}


# Function to perform sentiment prediction using the fine-tuned model
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():  # Disable gradient calculations for inference
        outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1).item()
    return reverse_label_map[predictions]

In [19]:
# Get random samples from test_df
num_samples = 10
random_samples = test_df.sample(num_samples)


# Make predictions and display results
print("\n--- Inference Results ---")
for index, row in random_samples.iterrows():
    text = row["text"]
    actual_label = reverse_label_map[row["labels"]]
    predicted_label = predict_sentiment(text)
    print(f"Text: {text}")
    print(f"Actual Sentiment: {actual_label}")
    print(f"Predicted Sentiment: {predicted_label}")
    print("-" * 50)


--- Inference Results ---
Text: জামায়াতে ইসলামীর নামব্যানার ব্যবহার করে কেউ দুর্বৃত্তপনা করলে সুনির্দিষ্ট তথ্য দিয়ে সহযোগিতা করতে হিন্দু সম্প্রদায়ের নেতাদের অনুরোধ করেছেন দলটির আমির শফিকুর রহমান
Actual Sentiment: hope
Predicted Sentiment: hope
--------------------------------------------------
Text: গাছের ডাব থেকে লেপতোষক গণভবনে কিছুই অবশিষ্ট নেই
Actual Sentiment: outrage
Predicted Sentiment: despair
--------------------------------------------------
Text: চট্টগ্রামে শিক্ষামন্ত্রীর বাসায় হামলা সংসদ সদস্যের কার্যালয়ে আগুন
Actual Sentiment: outrage
Predicted Sentiment: despair
--------------------------------------------------
Text: উৎপাদনস্থলেও কম দামে কাঁচামাল বিক্রি হচ্ছে আবার পণ্য পরিবহনে গত কয়েক দিনে ব্যবসায়ীদের কোনো চাঁদা দিতে হচ্ছে না
Actual Sentiment: hope
Predicted Sentiment: hope
--------------------------------------------------
Text: প্রবাসী আয় বেড়েছে সাঁইত্রিশ শতাংশ
Actual Sentiment: hope
Predicted Sentiment: despair
--------------------------------------------------
Text: 